In [100]:
import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point

In [61]:
df_subway_ridership_100 = pd.read_csv('../../data/raw/subway_turnstile/MTA_Subway_Hourly_Ridership__Beginning_February_2022_20240324.csv',
                                       nrows=100)

df_subway_ridership_iter = pd.read_csv('../../data/raw/subway_turnstile/MTA_Subway_Hourly_Ridership__Beginning_February_2022_20240324.csv', chunksize=10000)

In [62]:
df_subway_ridership_100

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,Georeference
0,03/30/2023 08:00:00 PM,subway,176,Lafayette Av (C),Brooklyn,metrocard,Metrocard - Fair Fare,6,0,40.686111,-73.973946,POINT (-73.97394561767578 40.68611145019531)
1,05/03/2023 02:00:00 AM,subway,176,Lafayette Av (C),Brooklyn,metrocard,Metrocard - Other,2,0,40.686111,-73.973946,POINT (-73.97394561767578 40.68611145019531)
2,03/16/2023 12:00:00 AM,subway,176,Lafayette Av (C),Brooklyn,metrocard,Metrocard - Full Fare,3,0,40.686111,-73.973946,POINT (-73.97394561767578 40.68611145019531)
3,04/29/2023 01:00:00 PM,subway,176,Lafayette Av (C),Brooklyn,metrocard,Metrocard - Full Fare,31,0,40.686111,-73.973946,POINT (-73.97394561767578 40.68611145019531)
4,06/26/2023 06:00:00 AM,subway,176,Lafayette Av (C),Brooklyn,metrocard,Metrocard - Full Fare,7,1,40.686111,-73.973946,POINT (-73.97394561767578 40.68611145019531)
...,...,...,...,...,...,...,...,...,...,...,...,...
95,05/27/2023 11:00:00 AM,subway,176,Lafayette Av (C),Brooklyn,metrocard,Metrocard - Other,6,0,40.686111,-73.973946,POINT (-73.97394561767578 40.68611145019531)
96,05/28/2023 12:00:00 PM,subway,176,Lafayette Av (C),Brooklyn,metrocard,Metrocard - Fair Fare,4,0,40.686111,-73.973946,POINT (-73.97394561767578 40.68611145019531)
97,06/13/2023 10:00:00 AM,subway,176,Lafayette Av (C),Brooklyn,metrocard,Metrocard - Other,5,0,40.686111,-73.973946,POINT (-73.97394561767578 40.68611145019531)
98,08/14/2023 03:00:00 PM,subway,176,Lafayette Av (C),Brooklyn,metrocard,Metrocard - Fair Fare,9,0,40.686111,-73.973946,POINT (-73.97394561767578 40.68611145019531)


In [63]:
list_df_2023 = []
for chunk in df_subway_ridership_iter:
    df_tmp = chunk.loc[chunk.loc[:,'borough'] == 'Manhattan']
    df_tmp.loc[:,'transit_timestamp'] = pd.to_datetime(df_tmp.loc[:,'transit_timestamp'])
    df_tmp = df_tmp.loc[df_tmp.loc[:,'transit_timestamp'].dt.year == 2023]
    df_tmp = df_tmp.loc[df_tmp.loc[:,'transit_mode'] == 'subway']
    df_tmp = df_tmp.drop(columns=['borough','Georeference','station_complex'])
    list_df_2023.append(df_tmp)

df_2023 = pd.concat(list_df_2023, ignore_index=True)

C:\Users\seung\AppData\Local\Temp\ipykernel_3424\3423417763.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp.loc[:,'transit_timestamp'] = pd.to_datetime(df_tmp.loc[:,'transit_timestamp'])
C:\Users\seung\AppData\Local\Temp\ipykernel_3424\3423417763.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_tmp.loc[:,'transit_timestamp'] = pd.to_datetime(df_tmp.loc[:,'transit_timestamp'])
C:\Users\seung\AppData\Local\Temp\ipykernel_3424\3423417763.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [68]:
df_2023.loc[:,'transit_hour'] = df_2023.loc[:,'transit_timestamp'].dt.hour
df_2023.loc[:,'station_complex_id'] = df_2023.loc[:,'station_complex_id'].astype(int)

C:\Users\seung\AppData\Local\Temp\ipykernel_3424\55842973.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_2023.loc[:,'station_complex_id'] = df_2023.loc[:,'station_complex_id'].astype(int)


In [87]:
df_2023_stations = df_2023.loc[:,['station_complex_id','latitude','longitude']].groupby('station_complex_id', as_index=False).mean()

In [88]:
df_2023_stations

,station_complex_id,latitude,longitude
0,8,40.764812,-73.973350
1,9,40.764664,-73.980660
2,10,40.759899,-73.984139
3,13,40.745495,-73.988692
4,14,40.741302,-73.989341
...,...,...,...
116,623,40.719500,-74.000083
117,624,40.713967,-74.008685
118,625,40.718605,-73.987494
119,628,40.710345,-74.006717


In [124]:
df_2023_agg = df_2023.drop(['fare_class_category',
                            'transit_mode',
                            'payment_method',
                            'transit_timestamp',
                            'latitude',
                            'longitude',
                            'transfers'], axis=1).groupby(['station_complex_id','transit_hour'], as_index=False).mean()

In [125]:
df_2023_agg

,station_complex_id,transit_hour,ridership
0,8,0,13.449488
1,8,1,4.981314
2,8,2,2.752800
3,8,3,2.013058
4,8,4,1.747275
...,...,...,...
2899,635,19,86.814561
2900,635,20,59.855121
2901,635,21,47.877675
2902,635,22,36.294161


In [126]:
def categorize_time(hour):
    if hour in range(0,6):
        return 'ridership_late_night'
    elif hour in range(6,10):
        return 'ridership_morning'
    elif hour in range(10,16):
        return 'ridership_midday'
    elif hour in range(16,20):
        return 'ridership_evening'
    else:
        return 'ridership_night'

In [127]:
df_2023_agg.loc[:,'time_category'] = df_2023_agg.loc[:,'transit_hour'].apply(categorize_time)

In [128]:
df_2023_agg = df_2023_agg.groupby(['station_complex_id','time_category'], as_index=False).sum().drop(['transit_hour'], axis=1)

In [133]:
df_2023_agg_pivot = df_2023_agg.pivot(index='station_complex_id', columns='time_category', values='ridership')

In [134]:
df_2023_agg_pivot = df_2023_agg_pivot.reset_index()
df_2023_agg_pivot.columns.name = ''

In [135]:
df_2023_agg_pivot

,station_complex_id,ridership_evening,ridership_late_night,ridership_midday,ridership_morning,ridership_night
0,8,549.074645,27.316380,379.283893,53.057698,167.351045
1,9,969.844061,85.678771,720.965483,235.273627,483.069353
2,10,700.162754,83.955955,483.170776,159.617568,418.384892
3,13,340.001318,55.609882,276.713270,92.005643,182.182819
4,14,559.011156,43.239404,460.629268,125.080793,217.280079
...,...,...,...,...,...,...
116,623,1404.683240,87.599435,1091.702413,204.587419,552.709026
117,624,1729.159382,83.855930,1218.714397,497.784949,378.018906
118,625,657.336639,172.035015,693.312322,368.702667,456.631117
119,628,2296.975412,138.171362,1645.396811,875.979734,650.945844


In [136]:
df_2023_agg_pivot = df_2023_agg_pivot.merge(df_2023_stations, on='station_complex_id')

points = [Point(x,y) for x, y in zip(df_2023_agg_pivot.loc[:,'longitude'], df_2023_agg_pivot.loc[:,'latitude'])]

In [139]:
gdf_2023_agg = gpd.GeoDataFrame(df_2023_agg_pivot, geometry=points).drop(columns=['latitude','longitude'])

In [140]:
gdf_2023_agg

,station_complex_id,ridership_evening,ridership_late_night,ridership_midday,ridership_morning,ridership_night,geometry
0,8,549.074645,27.316380,379.283893,53.057698,167.351045,POINT (-73.97335 40.76481)
1,9,969.844061,85.678771,720.965483,235.273627,483.069353,POINT (-73.98066 40.76466)
2,10,700.162754,83.955955,483.170776,159.617568,418.384892,POINT (-73.98414 40.75990)
3,13,340.001318,55.609882,276.713270,92.005643,182.182819,POINT (-73.98869 40.74549)
4,14,559.011156,43.239404,460.629268,125.080793,217.280079,POINT (-73.98934 40.74130)
...,...,...,...,...,...,...,...
116,623,1404.683240,87.599435,1091.702413,204.587419,552.709026,POINT (-74.00008 40.71950)
117,624,1729.159382,83.855930,1218.714397,497.784949,378.018906,POINT (-74.00868 40.71397)
118,625,657.336639,172.035015,693.312322,368.702667,456.631117,POINT (-73.98749 40.71860)
119,628,2296.975412,138.171362,1645.396811,875.979734,650.945844,POINT (-74.00672 40.71035)


In [141]:
gdf_2023_agg.to_file('../../data/processed/traffic/2023_subway_ridership.geojson', driver='GeoJSON')